In [1]:
def database():
	import pandas as pd
	from pycmtensor import Database
	sm = pd.read_csv("data/swissmetro.dat", sep="\t")
	db = Database(name="swissmetro", pandasDatabase=sm, choiceVar="CHOICE")
	globals().update(db.variables)
	exclude = ((PURPOSE != 1) * (PURPOSE != 3) + (CHOICE == 0)) > 0
	db.remove(exclude)
	db.data["CHOICE"] -= 1 # set the first choice to 0
	db.choices = sorted(db.data["CHOICE"].unique()) # save original choices
	db.autoscale(variables=['TRAIN_CO', 'TRAIN_TT', 'CAR_CO', 'CAR_TT', 
		'SM_CO', 'SM_TT'], default=100., verbose=False)
	return db

In [2]:
def betas():
	from pycmtensor.expressions import Beta
	b = {
		"b_cost": Beta("b_cost", 0.0, None, None, 0),
		"b_time": Beta("b_time", 0.0, None, None, 0),
		"asc_train": Beta("asc_train", 0.0, None, None, 0),
		"asc_car": Beta("asc_car", 0.0, None, None, 0),
		"asc_sm": Beta("asc_sm", 0.0, None, None, 1),
	}
	return b

def betas_fault():
	from pycmtensor.expressions import Beta
	b = {
		"b_cost": Beta("b_cost", 0.0, None, None, 0),
		"b_time": Beta("b_cost", 0.0, None, None, 0),
		"asc_train": Beta("asc_train", 0.0, None, None, 0),
		"asc_car": Beta("asc_car", 0.0, None, None, 0),
		"asc_sm": Beta("asc_sm", 0.0, None, None, 1),
	}
	return b
	

In [4]:
from pycmtensor.models import MNLogit
db = database()
locals().update(betas())
U_1 = b_cost * db["TRAIN_CO"] + b_time * db["TRAIN_TT"]
U_2 = b_cost * db["SM_CO"] + b_time * db["SM_TT"]
U_3 = b_cost * db["CAR_CO"] + b_time * db["CAR_TT"]
U = [U_1, U_2, U_3]
AV = [db["TRAIN_AV"], db["SM_AV"], db["CAR_AV"]]
mymodel = MNLogit(u=U, av=AV, database=db, name="mymodel")
mymodel.add_params(locals())

[2022-03-08 16:38:11] WARNING: Unused Betas from computational graph: asc_train asc_car removed. To explicity keep params in model, set param status=1.


In [ ]:
mymodel.params

[b_cost(0.0, TensorSharedVariable), b_time(0.0, TensorSharedVariable)]